In [29]:
import re
import numpy as np
import pandas as pd
import requests
import ete3
ncbi = ete3.NCBITaxa()

In [30]:
species='Francisella tularensis'

In [21]:
Int_DB = pd.read_table("/Users/thesis/Desktop/Google Drive/thesis-lorenz/Work in progress.../Databases/Extracted datasets/"+str(species), usecols=lambda x:x !="Unnamed: 0")

In [3]:
BASE = 'http://www.uniprot.org'
KB_ENDPOINT = '/uniprot/'
TOOL_ENDPOINT = '/uploadlists/'

In [4]:
taxid=ncbi.get_name_translator([species])[species][0]

def proteome_query(taxon):
    payload = {'query': 'taxonomy:'+str(taxon),
           'format': 'list'}

    result2 = requests.get(BASE + KB_ENDPOINT, params=payload)
    

    if result2.ok:
        output = set(result2.text.split())
        return output
    else:
        print('Something went wrong ', result.status_code)

prot_list = proteome_query(taxid)

#zelfde code als hierboven maar dan voor de grote file
with open ('/Volumes/Biomina_D2/thesis-lorenz/protein2ipr.dat','r') as f:
    with open ('/Volumes/Biomina_D2/thesis-lorenz/ipr_' + species+ '_out.txt','w') as w:
        outer_counter = 0
        for line in f:
            outer_counter += 1
            if outer_counter % 10000000 == 0:
                print('Processed',outer_counter,'lines')
            l = line.split('\t')
            if l[0] in prot_list:
                w.write(line)
            

In [31]:
#de code in de cellen vanaf de deze was die om de dictionaries te maken
#Ik laat die nog staan omdat die mss nog handig kunnen zijn
IPR_entry=[]
P2IPR_full={}
with open ('/Users/thesis/Desktop/ipr_' + str(species)+ '_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_full[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[] 
            IPR_entry.append(l[0])
        IPR_entry.append((l[1],l[2]))
        
for key in P2IPR_full.keys():
    P2IPR_full[key]=list(pd.unique(P2IPR_full[key]))

In [32]:
IPR_entry=[]
P2IPR_ID={}
with open ('/Users/thesis/Desktop/ipr_' + str(species)+ '_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_ID[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[]
            IPR_entry.append(l[0])
        IPR_entry.append(l[1])
for key in P2IPR_ID.keys():
    P2IPR_ID[key]=list(pd.unique(P2IPR_ID[key]))     

In [33]:
IPR_entry=[]
P2IPR_name={}
with open ('/Users/thesis/Desktop/ipr_' + str(species)+ '_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_name[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[]
            IPR_entry.append(l[0])
        IPR_entry.append(l[2])
        
for key in P2IPR_name.keys():
    P2IPR_name[key]=list(pd.unique(P2IPR_name[key]))

In [34]:
def Find_terms(ID):
    if ID in P2IPR_ID.keys():
        return P2IPR_ID[ID]
    else:
        return None
def Find_proteins(IPR):
    outp = []
    for key in P2IPR_ID:
        if IPR in P2IPR_ID[key]:
            outp.append(key)
    return outp

In [28]:
Int_DB['IPR_A']=Int_DB.Protein_A.apply(lambda x: Find_terms(x))

In [35]:
IPR_entry=[]
P2IPR_full_hum={}
with open ('/Users/thesis/Desktop/ipr_human_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_full_hum[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[] 
            IPR_entry.append(l[0])
        IPR_entry.append((l[1],l[2]))
        
for key in P2IPR_full_hum.keys():
    P2IPR_full_hum[key]=list(pd.unique(P2IPR_full_hum[key]))

In [36]:
IPR_entry=[]
P2IPR_ID_hum={}
with open ('/Users/thesis/Desktop/ipr_human_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_ID_hum[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[]
            IPR_entry.append(l[0])
        IPR_entry.append(l[1])
for key in P2IPR_ID_hum.keys():
    P2IPR_ID_hum[key]=list(pd.unique(P2IPR_ID_hum[key]))    

In [37]:
IPR_entry=[]
P2IPR_name_hum={}
with open ('/Users/thesis/Desktop/ipr_human_out.txt','r') as f:
    for line in f:
        l = line.split('\t')
        if l[0] not in IPR_entry:
            if IPR_entry != []:
                P2IPR_name_hum[IPR_entry[0]]=IPR_entry[1:]
                IPR_entry =[]
            IPR_entry.append(l[0])
        IPR_entry.append(l[2])
        
for key in P2IPR_name_hum.keys():
    P2IPR_name_hum[key]=list(pd.unique(P2IPR_name_hum[key]))

In [38]:
def Find_terms_hum(ID):
    if ID in P2IPR_ID_hum.keys():
        return P2IPR_ID_hum[ID]
    else:
        return None
def Find_proteins_hum(IPR):
    outp = []
    for key in P2IPR_ID_hum:
        if IPR in P2IPR_ID_hum[key]:
            outp.append(key)
    return outp

In [33]:
Int_DB['IPR_B']=Int_DB.Protein_B.apply(lambda x: Find_terms_hum(x))

In [15]:
Int_DB.to_csv("/Users/thesis/Desktop/Int_DB_IPR_" + species, sep ="\t")

In [39]:
Int_DB = pd.read_table('/Users/thesis/Desktop/Int_DB_IPR_GO2_'+ species,usecols=lambda x:x !="Unnamed: 0")

In [40]:
sel1 = Int_DB.IPR_A.isnull()
sel2 = Int_DB.IPR_B.isnull()

In [41]:
Int_DB.loc[sel1,'IPR_A'] = Int_DB.loc[sel1,'Protein_A'].apply(lambda x: Find_terms(x))
Int_DB.loc[sel2,'IPR_B'] = Int_DB.loc[sel2,'Protein_B'].apply(lambda x: Find_terms_hum(x))

In [42]:
Int_DB.to_csv("/Users/thesis/Desktop/Int_DB_IPR2_GO2_" + species, sep ="\t")